# Make the photometry of the companion star

- author : Sylvie Dagoret
- date June 17 2019

Use Photutils

In [1]:
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import re

import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates
from matplotlib import gridspec
%matplotlib inline

import numpy as np

from scipy.interpolate import interp1d

from astropy.time import Time

In [2]:
from astropy.stats import sigma_clipped_stats
from photutils import CircularAperture
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils import DAOStarFinder
from photutils import make_source_mask
from astropy.stats import SigmaClip
from photutils import Background2D, MedianBackground

ModuleNotFoundError: No module named 'photutils'

In [ ]:
if not 'workbookDir' in globals():
    workbookDir = os.getcwd()
print('workbookDir: ' + workbookDir)

spectractordir=workbookDir+"/../../Spectractor"
print('spectractordir: ' + spectractordir)
toolsdir=workbookDir+"/../common_tools"
print("toolsdir:",toolsdir)


import sys
sys.path.append(workbookDir)
sys.path.append(spectractordir)
sys.path.append(os.path.dirname(workbookDir))
sys.path.append(toolsdir)



from spectractor import parameters
from spectractor.extractor.extractor import Spectractor
from spectractor.logbook import LogBook
from spectractor.extractor.dispersers import *
from spectractor.extractor.spectrum import *
from spectractor.tools import ensure_dir

from libatmscattering import *


plt.rcParams["axes.labelsize"]="large"
plt.rcParams["axes.linewidth"]=2.0
plt.rcParams["xtick.major.size"]=8
plt.rcParams["ytick.major.size"]=8
plt.rcParams["ytick.minor.size"]=5
plt.rcParams["xtick.labelsize"]="large"
plt.rcParams["ytick.labelsize"]="large"

plt.rcParams["figure.figsize"]=(15,15)
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.titleweight'] = 'bold'
#plt.rcParams['axes.facecolor'] = 'blue'
plt.rcParams['xtick.direction'] = 'out'
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['lines.markeredgewidth'] = 0.3 # the line width around the marker symbol
plt.rcParams['lines.markersize'] = 5  # markersize, in points
plt.rcParams['grid.alpha'] = 0.75 # transparency, between 0.0 and 1.0
plt.rcParams['grid.linestyle'] = '-' # simple line
plt.rcParams['grid.linewidth'] = 0.4 # in points



In [ ]:
######### CONFIGURATION

## wavelength
WLMIN = 380.0
WLMAX = 1000.0
#NBWLBIN = 62
NBWLBIN = 124
WLBINWIDTH = (WLMAX - WLMIN) / float(NBWLBIN)

WLMINBIN = np.arange(WLMIN, WLMAX, WLBINWIDTH)
WLMAXBIN = np.arange(WLMIN + WLBINWIDTH, WLMAX + WLBINWIDTH, WLBINWIDTH)
WLMEANBIN=(WLMINBIN + WLMAXBIN)/2.



print('WLMINBIN..................................=', WLMINBIN.shape, WLMINBIN)
print('WLMAXBIN..................................=', WLMAXBIN.shape, WLMAXBIN)
print('NBWLBIN...................................=', NBWLBIN)
print('WLBINWIDTH................................=', WLBINWIDTH)

## colors

# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NBWLBIN)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NBWLBIN), alpha=1)

## output directory for tables
#ouputtabledir="outputtabledir"

## create output directory
#ensure_dir(ouputtabledir)

In [ ]:
#---------------------------------------------------------------------
def GetWLBin(wl):
    """

    :param wl: wavelength scalar
    :return: index
    """

    set_ibin = np.where(np.logical_and(WLMINBIN <= wl, WLMAXBIN > wl))[0]

    if len(set_ibin)==1:
        return set_ibin[0]
    else:
        return -1
#---------------------------------------------------------------------
def GETWLLabels():

    all_labels=[]
    for idx in np.arange(NBWLBIN):
        label="$\lambda$ : {:3.0f}-{:3.0f} nm".format(WLMINBIN[idx], WLMAXBIN[idx])
        all_labels.append(label)
    all_labels=np.array(all_labels)
    return all_labels
#------------------------------------------------------------------------

WLLABELS=GETWLLabels()

In [ ]:
def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    
    For example for the PSF
    
    x=pixel number
    y=Intensity in pixel
    
    values-x
    weights=y=f(x)
    
    """
    average = np.average(values, weights=weights)
    variance = np.average((values - average) ** 2, weights=weights)  # Fast and numerically precise
    return average, np.sqrt(variance)


In [ ]:
def ComputeStarPhotometry(image):
    """

    :param image:
    :return:
    """

    mask = make_source_mask(image, snr=3, npixels=5, dilate_size=11)
    mean, median, std = sigma_clipped_stats(image, sigma=3.0, mask=mask)
    sigma_clip = SigmaClip(sigma=3.)
    bkg_estimator = MedianBackground()
    bkg = Background2D(image, (50, 50), filter_size=(3, 3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
    signal = image - bkg.background
    mean, median, std = sigma_clipped_stats(signal, sigma=3.0)
    daofind = DAOStarFinder(fwhm=10.0, threshold=100. * std)
    sources = daofind(signal - median)
    for col in sources.colnames:
        sources[col].info.format = '%.8g'  # for consistent table output
    print(sources)

    #select the source having ymin
    allY=sources["ycentroid"]
    idx=np.where(allY==allY.min())[0][0]
    flux=sources["flux"][idx]
    mag=sources["mag"][idx]
    
    print(" idx={}".format(idx))
    
    return flux,mag

In [ ]:
# where are the spectra
#----------------------
thedate = "20190215"
#input_directory = "/Users/dagoret/DATA/PicDuMidiFev2019/spectractor_output_prod3/" + thedate
input_directory = "/Users/dagoret/DATA/PicDuMidiFev2019/spectractor_output_prod4/" + thedate
rawinput_directory="/Users/dagoret/DATA/PicDuMidiFev2019/prod_"+thedate+"_v4"

In [ ]:
#-------------------------------------------------------------------------------------
def GetAllFiles(dir):
    """
    GetAllFiles(dir): provides the list of relevant files inside the directory dir


    :param dir: input directory pathname
    :return: list of files
    """



    # get all files
    onlyfiles = [f for f in listdir(dir) if isfile(join(dir, f))]
    onlyfiles = np.array(onlyfiles)

    # sort files
    sortedindexes = np.argsort(onlyfiles)
    onlyfiles = onlyfiles[sortedindexes]

    # get only _spectrum.fits file
    onlyfilesspectrum = []
    for file_name in onlyfiles:
        if re.search("^T.*_spectrum.fits$", file_name):
            # check if other files exits

            filetype = file_name.split('.')[-1]

            output_filename = os.path.basename(file_name)
            output_filename = os.path.join(dir, output_filename)
            output_filename_spectrogram = output_filename.replace('spectrum', 'spectrogram')
            output_filename_psf = output_filename.replace('spectrum.fits', 'table.csv')

            # go to next simulation if output files already exists
            if os.path.isfile(output_filename) and os.path.isfile(output_filename_spectrogram) and os.path.isfile(
                    output_filename_psf):
                filesize = os.stat(output_filename).st_size
                print(">>>>> output filename : {} already exists with size {} ".format(output_filename, filesize))

                filesize = os.stat(output_filename_spectrogram).st_size
                print(">>>>> output filename : {} already exists with size {} ".format(output_filename_spectrogram,
                                                                                       filesize))

                filesize = os.stat(output_filename_psf).st_size
                print(">>>>> output filename : {} already exists with size {} ".format(output_filename_psf, filesize))

                onlyfilesspectrum.append(re.findall("(^T.*_spectrum.fits$)", file_name)[0])

    # sort again all the files
    onlyfilesspectrum = np.array(onlyfilesspectrum)
    sortedindexes = np.argsort(onlyfilesspectrum)
    onlyfilesspectrum = onlyfilesspectrum[sortedindexes]

    # get basnemae of files for later use (to check if _table.csv and _spectrogram.fits exists
    onlyfilesbasename = []
    for f in onlyfilesspectrum:
        onlyfilesbasename.append(re.findall("(^T.*)_spectrum.fits$", f)[0])


    return onlyfilesspectrum,onlyfilesbasename

In [ ]:
def ReadAllFiles(dir, filelist):
    """

    ReadAllFiles(dir, filelist): Read all files

    :param dir: input directory
    :param filelist: list fo files
    :return: various containers
    """


    # init all container
    all_indexes=[]   # continuous index
    all_eventnum=[]  # event number on filename

    all_airmass=[]   #airmass

    all_lambdas = [] # wavelength

    #fluxes
    all_flux=[]
    all_errflux=[]


    # magnitudes
    all_mag=[]
    all_errmag=[]

    # absorption (Magnitude corrected from Rayleigh
    all_abs=[]
    all_errabs=[]


    all_dt       = []   # time since beginning in hours
    all_datetime = []   # datetime

    # preselection flag
    all_flag = []


    #bad indexes and filename
    all_badidx=[]
    all_badfn=[]


    all_BGimg=[]
    all_Rawimg=[]

    NBSPEC = len(filelist)


    #----------------------------------
    # Extract spectra information from files
    # compute magnitudes
    #-----------------------------

    count=-1  #counter of good files before filtering

    # loop on all files
    for idx in np.arange(0, NBSPEC):
        if idx==322:
            print("SKIP bad file",filelist[idx] )
            continue

        count+=1

        theeventnum = int(filelist[idx].split(".")[1].split("_")[0])


        print(" read {}) : event {} , file {}".format(idx,theeventnum,filelist[idx]))


        fullfilename = os.path.join(dir, filelist[idx])
        fullfilename_spectrogram = fullfilename.replace('spectrum', 'spectrogram')
        rawfilename = filelist[idx].replace('_spectrum.fits', '.fit')
        fillrawfilename=os.path.join(rawinput_directory,rawfilename)
        

        #try:
        if 1:
            # read fits file
            hdu = fits.open(fullfilename)
            hdu2=fits.open(fullfilename_spectrogram)
            hdu3=fits.open(fillrawfilename)

            

            # decode header
            header=hdu[0].header

            am=header["AIRMASS"]
            date=header["DATE-OBS"]

            if idx==0:
                T0=t = Time(date, format='isot', scale='utc')
            T=Time(date, format='isot', scale='utc')
            thedatetime=T.to_datetime()

            DT=(T-T0).sec/(3600.0)  # convert in hours


            # decode data
            data=hdu[0].data

            # extract wavelength, spectrum and errors
            wavelength=data[0,:]
            spec = data[1,:]
            err=data[2,: ]

            if(len(wavelength)==0 or len(spec)==0 or len(err)==0):
                print(">>>>>>>>>>>>>>  Empty file ",idx,")::",onlyfilesspectrum[idx] )
                print(">>>>>>>>>>>>>>>>>>>>>>>>>>>",len(wavelength)," , ",len(spec0), " , ",len(err))



            # sort the wavelengths
            wl_sorted_idx=np.argsort(wavelength)

            wl=wavelength[wl_sorted_idx]
            fl=spec[wl_sorted_idx]
            errfl=err[wl_sorted_idx]
            wlbins = [GetWLBin(w) for w in wl]
            wlbins = np.array(wlbins)


            # defines good measurmements as flux >0 and wavelength in selected bons
            goodpoints=np.where(np.logical_and(fl != 0, wlbins != -1))

            if(len(goodpoints)==0):
                print(">>>>>>>>>>>>>>  No Good points  ", idx, ")::", onlyfilesspectrum[idx])
                print(">>>>>>>>>>>>>>  No Good points  ", "wl = ",wl)
                print(">>>>>>>>>>>>>>  No Good points  ", "fl = ", fl)
                print(">>>>>>>>>>>>>>  No Good points  ", "errfl = ", errfl)



            # keep good points (wl,flux)
            wl=wl[goodpoints]
            fl=fl[goodpoints]
            errfl=errfl[goodpoints]


            # convert flux into magnitudes for each wavelength
            mag = -2.5 * np.log10(fl)
            errmag = errfl/fl

            #compute effective slant Rayleigh optical depth (not the vertical optical depth)
            od_adiab = RayOptDepth_adiabatic(wl,altitude=2890.5, costh=1./am)  # Rayleigh optical depth

            # absorption magnitude corrected from Rayleigh attenuation
            abs=mag-2.5/np.log(10.)*od_adiab
            errabs=errmag





            # save for each observation  { event-id, airmass, set of points (wl,flux,errflux, mag,abs,errabs) }
            if(len(mag>0)):
                #all_indexes.append(idx)
                all_indexes.append(count)
                all_eventnum.append(theeventnum)
                all_airmass.append(am)
                all_lambdas.append(wl)
                all_flux.append(fl)
                all_errflux.append(errfl)
                all_mag.append(mag)
                all_errmag.append(errmag)
                all_abs.append(abs)
                all_errabs.append(errabs)
                all_dt.append(DT)
                all_datetime.append(thedatetime)
                all_BGimg.append(hdu2[2].data)
                all_Rawimg.append(hdu3[0].data)

                absmin = abs.min()
                absmax = abs.max()


                # Set a quality flag
                if absmin < 25 or absmax > 32:
                    print("file index idx = ", count, "==>  filename = ", onlyfilesspectrum[idx], " absmin= ", absmin,
                          " absmax = ", absmax)
                    all_flag.append(False)
                    all_badidx.append(count)
                    all_badfn.append(onlyfilesspectrum[idx])
                else:
                    all_flag.append(True)



        #except:
        if 0:
            print("Unexpected error:", sys.exc_info()[0])
            pass

    # transform container in numpy arrays
    all_indexes=np.array(all_indexes)
    all_eventnum = np.array(all_eventnum)

    all_airmass = np.array(all_airmass)

    all_lambdas = np.array(all_lambdas)

    all_flux = np.array(all_flux)
    all_errflux = np.array(all_errflux)


    all_mag=np.array(all_mag)
    all_errmag=np.array(all_errmag)

    all_abs = np.array(all_abs)
    all_errabs = np.array(all_errabs)

    all_dt = np.array(all_dt)
    all_datetime=np.array(all_datetime)

    all_flag = np.array(all_flag)

    all_badidx=np.array(all_badidx)
    all_badfn=np.array(all_badfn)


    return all_indexes,all_eventnum,all_airmass,all_lambdas,all_flux,all_errflux,all_mag,all_errmag,all_abs,all_errabs,all_dt,all_datetime,all_flag,all_badidx,all_badfn, all_BGimg, all_Rawimg


In [ ]:
#############################################
# 1) Get list of files
##########################################
onlyfilesspectrum,onlyfilesbasename=GetAllFiles(input_directory)



In [ ]:
#############################################
# 2) Read fits file
##########################################


NBSPEC = len(onlyfilesspectrum)

print('NBSPEC....................................= ', NBSPEC)


all_indexes, all_eventnum, all_airmass, all_lambdas, all_flux, all_errflux, all_mag, all_errmag, all_abs, all_errabs, all_dt, all_datetime, all_flag, all_badidx, all_badfn, all_BGimg, all_Rawimg=ReadAllFiles(input_directory, onlyfilesspectrum)


In [ ]:
NBSPEC

# Compute Photometry

In [ ]:
all_mag=[]
all_flux=[]

idx=0
for img in all_Rawimg:
    print("************************************** {} ***************************************".format(idx))
    flux,mag=ComputeStarPhotometry(img)

    all_mag.append(mag)
    all_flux.append(flux)
    idx+=1
    
    if idx>50:
        break

all_mag=np.array(all_mag)
all_flux=np.array(all_flux)


print(all_mag)

In [ ]:
assert False

# Study a particular image

## Selection

In [ ]:
index=32 # exemple de bougé
#index=0 # image OK
image=all_Rawimg[index]
vmin=image.min()
vmax=image.max()/50.
thefilename=onlyfilesspectrum[index] #onlyfilesbasename

In [ ]:
data=image

## Show raw image

In [ ]:
img=plt.imshow(data,origin="lower",vmin=vmin,vmax=vmax,cmap="jet")
plt.grid(color="w")
plt.title(thefilename)
plt.xlabel(" X - pixel ")
plt.ylabel(" Y - pixel ")
plt.colorbar(img,orientation="horizontal")

## Plot histogram

In [ ]:
plt.figure(figsize=(12,8))
XMIN=10
XMAX=1000
plt.hist(data.flatten(), bins = 10 ** np.linspace(np.log10(XMIN), np.log10(XMAX), 100),color="b",alpha=0.8)
plt.gca().set_xscale("log")
#plt.gca().set_yscale("log")
plt.grid()
plt.title("Reduced image ADU counts for {}".format(thefilename))
plt.xlabel("ADU counts")
plt.ylabel("number of pixels")

In [ ]:
mean, median, std = sigma_clipped_stats(data, sigma=3.0)    
print((mean, median, std))    

## Search Sources in raw image

In [ ]:
daofind = DAOStarFinder(fwhm=10.0, threshold=100.*std)    
sources = daofind(data - median) 

In [ ]:
for col in sources.colnames:    
    sources[col].info.format = '%.8g'  # for consistent table output
print(sources)    

In [ ]:
sources

In [ ]:
positions = (sources['xcentroid'], sources['ycentroid'])
apertures = CircularAperture(positions, r=50.)
norm = ImageNormalize(stretch=SqrtStretch())
#plt.imshow(data,cmap='Greys', origin='lower', norm=norm)
img=plt.imshow(data,cmap='jet', origin='lower', norm=norm,vmin=vmin,vmax=vmax)
apertures.plot(color='red', lw=3, alpha=1.0)

plt.grid(color="w")
plt.title(thefilename)
plt.colorbar(img,orientation="horizontal")
plt.xlabel(" X - pixel ")
plt.ylabel(" Y - pixel ")

## Background extraction

In [ ]:
mask = make_source_mask(data, snr=3, npixels=5, dilate_size=11)
mean, median, std = sigma_clipped_stats(data, sigma=3.0, mask=mask)
print((mean, median, std))  

In [ ]:
sigma_clip = SigmaClip(sigma=3.)
bkg_estimator = MedianBackground()
bkg = Background2D(data, (50, 50), filter_size=(3, 3),sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)

In [ ]:
#img=plt.imshow(bkg.background, origin='lower', cmap='Greys_r')
img=plt.imshow(bkg.background, origin='lower', cmap='jet')
plt.grid(color="w")
plt.title("Background for {}".format(thefilename))
plt.xlabel(" X - pixel ")
plt.ylabel(" Y - pixel ")
plt.colorbar(img,orientation="horizontal")

In [ ]:
plt.figure(figsize=(12,8))
XMIN=10
XMAX=1000
plt.hist(bkg.background.flatten(), bins = 10 ** np.linspace(np.log10(XMIN), np.log10(XMAX), 100),color="b",alpha=0.8)
plt.gca().set_xscale("log")
#plt.gca().set_yscale("log")
plt.grid()
plt.title("Background ADU counts for {}".format(thefilename))
plt.xlabel("ADU counts")
plt.ylabel("number of pixels")

In [ ]:
plt.figure(figsize=(12,8))
XMIN=20
XMAX=100
plt.hist(bkg.background.flatten(), bins = np.linspace(XMIN,XMAX, 100),color="b",alpha=0.8)
#plt.gca().set_xscale("log")
#plt.gca().set_yscale("log")
plt.grid()
plt.title("Background ADU counts for {}".format(thefilename))
plt.xlabel("ADU counts")
plt.ylabel("number of pixels")

In [ ]:
signal=data - bkg.background
vmin=signal.min()
vmax=signal.max()/50.
#plt.imshow(signal, norm=norm, origin='lower',cmap='Greys_r',vmin=vmin,vmax=vmax)
img=plt.imshow(signal, norm=norm, origin='lower',cmap='jet',vmin=vmin,vmax=vmax)
plt.colorbar(img,orientation="horizontal")
plt.title("Signal for {}".format(thefilename))
plt.xlabel(" X - pixel ")
plt.ylabel(" Y - pixel ")
plt.grid(color="w")

In [ ]:
mean, median, std = sigma_clipped_stats(signal, sigma=3.0)    
print((mean, median, std))    

In [ ]:
daofind = DAOStarFinder(fwhm=10.0, threshold=100.*std)    
sources = daofind(data - median) 

In [ ]:
for col in sources.colnames:    
    sources[col].info.format = '%.8g'  # for consistent table output
print(sources)   

In [ ]:
sources

In [ ]:
positions = (sources['xcentroid'], sources['ycentroid'])
apertures = CircularAperture(positions, r=50.)
norm = ImageNormalize(stretch=SqrtStretch())
#plt.imshow(data,cmap='Greys', origin='lower', norm=norm)
img=plt.imshow(signal,cmap='jet', origin='lower', norm=norm,vmin=vmin,vmax=vmax)
apertures.plot(color='red', lw=3, alpha=1.0)
plt.grid(color="w")
plt.title("Signal for {}".format(thefilename))
plt.colorbar(img,orientation="horizontal")
plt.xlabel(" X - pixel ")
plt.ylabel(" Y - pixel ")

In [ ]:
plt.figure(figsize=(12,8))
XMIN=0.01
XMAX=10000
plt.hist(signal.flatten(), bins = 10 ** np.linspace(np.log10(XMIN), np.log10(XMAX), 100),color="b",alpha=0.8)
plt.gca().set_xscale("log")
#plt.gca().set_yscale("log")
plt.grid()
plt.title("Signal ADU counts for {}".format(thefilename))
plt.xlabel("ADU counts")
plt.ylabel("number of pixels")

In [ ]:
x0=sources["xcentroid"][0]
y0=sources["ycentroid"][0]
siz=20
pixel_to_arcsec=0.159
extent=(-siz*pixel_to_arcsec,siz*pixel_to_arcsec, -siz*pixel_to_arcsec,siz*pixel_to_arcsec )

In [ ]:
vignette=signal[int(y0-siz):int(y0+siz),int(x0-siz):int(x0+siz)]

In [ ]:
plt.figure(figsize=(12,8))
XMIN=10
XMAX=100000
plt.hist(vignette.flatten(), bins = 10 ** np.linspace(np.log10(XMIN), np.log10(XMAX), 100),color="b",alpha=0.8)
plt.gca().set_xscale("log")
#plt.gca().set_yscale("log")
plt.grid()
plt.title("Star ADU counts for {}".format(thefilename))
plt.xlabel("ADU counts")
plt.ylabel("number of pixels")

In [ ]:
img=plt.imshow(vignette,origin="lower",cmap="jet",extent=extent)
plt.colorbar(img,orientation="horizontal")
plt.xlabel("$\\theta_X$ (arcsec)")
plt.ylabel("$\\theta_Y$ (arcsec)")
plt.title(thefilename)
plt.grid(color="white")

In [ ]:
histx=vignette.sum(axis=0)
histy=vignette.sum(axis=1)

In [ ]:
lx=len(histx)
ly=len(histy)
mx,sx=weighted_avg_and_std(np.arange(lx),histx)
my,sy=weighted_avg_and_std(np.arange(ly),histy)
hxmin,hxmax=histx.min(),histx.max()
hymin,hymax=histy.min(),histy.max()
hxaver=(hxmin+hxmax)/2.
hyaver=(hymin+hymax)/2.

In [ ]:
sx,sy

In [ ]:
plt.subplot(221)
plt.plot(histx,'b-o')
plt.plot([mx,mx],[hxmin,hxmax],"r-")
plt.plot([(lx+1.)/2.-2.36/2.*sx, (lx+1)/2.+2.36/2.*sx],[hxaver,hxaver],"g-")
plt.xlabel("x - pixel")
plt.title("$\sigma_x$= {:1.2f} pix = {:1.2f} arcsec".format(sx,sx*pixel_to_arcsec))
plt.grid()
plt.subplot(222)
plt.plot(histy,'b-o')
plt.plot([my,my],[hymin,hymax],"r-")
plt.plot([(ly+1)/2.-2.36/2.*sy, (ly+1)/2.+2.36/2.*sy],[hyaver,hyaver],"g-")
plt.xlabel("y - pixel")
plt.title("$\sigma_y$= {:1.2f} pix = {:1.2f} arcsec".format(sy,sy*pixel_to_arcsec))
plt.grid()
plt.suptitle(thefilename,fontsize=20)